## Welcome to Vibes Investigating with Louie!

This tutorial goes along with the Louie vibes investigation training. You can fork & use it with your same Louie credentials.

Further reading:

* Louie ReadtheDocs: https://louie.readthedocs.io/en/latest/

* Louie GitHub: https://github.com/graphistry/louie-py

* PyGraphistry ReadTheDocs: https://pygraphistry.readthedocs.io/en/latest/


## 1. Install

Get package `louieai` using pip or uv. Louie requires Python 3.10+ and Pandas/Arrow

It installs PyGraphistry 0.41+ for auth & graphs

In [52]:
!pip install -q louieai

## Import

In [1]:
import graphistry

import louieai

print("graphistry", graphistry.__version__, "louieai", louieai.__version__)

/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


graphistry 0.41.0 louieai 0.5.1


## Configure

1. Graphistry server: Holds your account, semantic layer, and adds GPU viz & remote GFQL querying

2. Louie server: Datathreads & agentic vibes


In [ ]:
LOUIE_SERVER_URL = "https://den.louie.ai"
GRAPHISTRY_DOMAIN = "hub.graphistry.com"

# MAKE API KEY: https://hub.graphistry.com/users/personal/key/
g = graphistry.register(
    api=3,
    server=GRAPHISTRY_DOMAIN,
    personal_key_id=FILL_ME_IN,
    personal_key_secret=FILL_ME_IN,
    org_name=FILL_ME_IN,
)


def new_thread(share_mode="Private"):
    return louieai(g, server_url=LOUIE_SERVER_URL, share_mode=share_mode)


lui = new_thread()

print("Connected!")

## Hello World

Louie is an agent that uses many other LLMs and agents. You can start just as if it is ChatGPT chat.

In [3]:
lui("""

sing me a song

""")

<LouieAI Notebook Interface | Session: Active | History: 1 responses | Traces: Disabled | Latest: 1 text>

In [4]:
lui("""

sing it again, but now in pig latin!

""")

<LouieAI Notebook Interface | Session: Active | History: 2 responses | Traces: Disabled | Latest: 1 text>

In [5]:
lui(
    """

back to normal, but now with traces!

""",
    traces=True,
)

<LouieAI Notebook Interface | Session: Active | History: 3 responses | Traces: Disabled | Latest: 1 text>

## Use `lui` as a chat cursor

The `lui` object indexes into your chat (notebook) and its artifacts

You can make new chats, and add to existing

In [6]:
lui.thread_id

'D_pkKkYkQsv_tQe0NscLl9'

In [22]:
# Save old
lui_old = lui

# Fresh thread with empty history
lui = new_thread()

lui("sing me a new song")
lui("sing me another song")

lui.thread_id

'D_kzETo4kBuV3F2wBBXBsK'

In [8]:
# Latest text element in current run
lui.text

"🎵 I'm Louie, the data hound, in the cyberpunk city, I'm renowned. With paws on the keyboard, I dig and I delve, solving data puzzles, I truly excel. From queries to graphs, I make them align, turning raw data into something divine. So let's embark on this data quest, with me by your side,"

In [9]:
# Latest text elements in its previous run
lui[-2].text

"🎵 Woof, woof, I'm Louie, the data dog, chasing data through the fog. With a wag of my tail and a bark so bright, I'll fetch the answers, day or night. Queries and graphs, I'll make them dance, turning data into a chance. So let's dive in, human friend, together we'll"

In [10]:
# Latest text element in previous session
lui_old.text

"🎶 Woof woof, I'm Louie, the data dog, chasing data through the fog. With a wag of my tail and a bark so bright, I'll fetch your insights day and night. From queries to graphs, I'll make it clear, turning data puzzles into cheer. So let's dive in, let's explore, with Louie by your side, you'll never be bored! 🎶\n\n*traces of data flow through the air, as Louie sniffs them out with flair. Each byte and bit, a trail to follow, leading to insights that are never hollow. With a keen eye and a wagging tail, Louie ensures your"

## Agents

### Databricks SQL Passthrough

Louie has databases connected and preindexed as a managed semantic layer. You can query it quickly by doing pushdown queries that skim LLMs, so more like tool calls.


In [ ]:
lui(
    """SELECT * FROM `client_demos`.`botsv3`.`o365_management_activity_flat_tcook`
    LIMIT 10""",
    agent="DatabricksPassthroughAgent",
)

In [28]:
## Quick reference of latest dataframe
lui.df.sample(3)

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
4,104.238.59.42,0a627e9e-f0d0-6000-daf9-dc8f468313e3,2018-08-20T13:06:50,SharePoint,9b5ad97e-f03f-4648-a536-08d5f28d5cad,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
8,199.66.91.253,08557e9e-2041-6000-daf9-d90e9f2c1023,2018-08-20T11:55:22,SharePoint,d7ee3d62-60cb-427b-b0db-08d5f26d99f1,No,File,67091393-e290-421e-ac6a-2734e2b12a94,57c5cd78-2a0f-42ee-8dab-afce305ec89e,https://frothly-my.sharepoint.com/personal/fyo...,...,tar,archive.tar,Documents,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
9,199.66.91.253,fc547e9e-f0c2-6000-1d75-39268176f5e6,2018-08-20T11:54:35,SharePoint,cf2ae00e-0efc-41e5-d0dc-08d5f26d7e46,Yes,File,76079ea8-0a58-414e-b493-8580089c8419,7ba794f0-38aa-45d8-8d2a-3c932e87e4f5,https://frothly-my.sharepoint.com/User Photos/...,...,jpg,fyodor_froth_ly_SThumb.jpg,User Photos/Profile Pictures,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,3b605151-ac0a-46ff-9e16-14e04be8a8a1,SharePoint


In [29]:
lui.df

,ClientIP,CorrelationId,CreationTime,EventSource,Id,ImplicitShare,ItemType,ListId,ListItemUniqueId,ObjectId,...,SourceFileExtension,SourceFileName,SourceRelativeUrl,UserAgent,UserId,UserKey,UserType,Version,WebId,Workload
0,107.77.213.173,9e627e9e-d0dd-6000-daf9-da44fcd45d4e,2018-08-20T13:16:56,SharePoint,8a1fd9ad-95d3-4bea-a806-08d5f28ec619,No,File,67091393-e290-421e-ac6a-2734e2b12a94,37ab8c26-f775-4a03-97b3-074c81a00f33,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,beverages-02-00034-v2.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
1,107.77.213.173,9e627e9e-60d3-6000-32f0-2235e1b3a20b,2018-08-20T13:16:56,SharePoint,7d1dd9e0-63b3-4277-7a03-08d5f28ec5e3,No,File,67091393-e290-421e-ac6a-2734e2b12a94,bb017930-2bf5-4953-b38a-716ba3217703,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,craftbeerdotcom-beer-styles.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
2,107.77.213.173,9e627e9e-a0db-6000-daf9-da5a21ed5a92,2018-08-20T13:16:56,SharePoint,f5ad6c89-25b3-420c-f889-08d5f28ec656,No,File,67091393-e290-421e-ac6a-2734e2b12a94,456e3291-27ad-455e-9cb7-a01722ffa0fa,https://frothly-my.sharepoint.com/personal/fyo...,...,pdf,fundamental of beer and hop chemistry.pdf,Documents,OneDriveMpc/1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
3,40.97.148.181,fa2fd17c-daf8-4062-8f87-d411eb537314,2018-08-20T13:16:54,SharePoint,0ad166f0-5312-4e35-e017-08d5f28ec4fb,None,Web,None,None,fa2fd17c-daf8-4062-8f87-d411eb537314,...,None,None,None,Substrate Search 1.0,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,None,SharePoint
4,104.238.59.42,0a627e9e-f0d0-6000-daf9-dc8f468313e3,2018-08-20T13:06:50,SharePoint,9b5ad97e-f03f-4648-a536-08d5f28d5cad,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
5,65.52.243.21,fb617e9e-c0c7-6000-32f0-2e462ff7bbde,2018-08-20T13:05:48,SharePoint,4b143e6d-91b0-4b07-7253-08d5f28d37c3,None,List,67091393-e290-421e-ac6a-2734e2b12a94,None,https://frothly-my.sharepoint.com/personal/pce...,...,None,None,None,ODMTADocCache/1.0,app@sharepoint,i:0i.t|00000003-0000-0ff1-ce00-000000000000|ap...,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
6,104.238.59.42,f9617e9e-f0de-6000-32f0-299be8e0a683,2018-08-20T13:05:40,SharePoint,a1cac3e4-3102-4e0c-d258-08d5f28d3331,No,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
7,104.238.59.42,f8617e9e-a0d4-6000-3667-381044fbe5d3,2018-08-20T13:05:36,SharePoint,7d6f73bd-f3c3-4c95-7b24-08d5f28d30de,None,File,67091393-e290-421e-ac6a-2734e2b12a94,d2c4bb13-c97e-4707-9e9b-53dc0e2513b5,https://frothly-my.sharepoint.com/personal/pce...,...,pptx,Beer styles.pptx,Documents,Microsoft Office PowerPoint 2014,pcerf@froth.ly,i:0h.f|membership|1003bffdac730049@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
8,199.66.91.253,08557e9e-2041-6000-daf9-d90e9f2c1023,2018-08-20T11:55:22,SharePoint,d7ee3d62-60cb-427b-b0db-08d5f26d99f1,No,File,67091393-e290-421e-ac6a-2734e2b12a94,57c5cd78-2a0f-42ee-8dab-afce305ec89e,https://frothly-my.sharepoint.com/personal/fyo...,...,tar,archive.tar,Documents,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,fyodor@froth.ly,i:0h.f|membership|1003bffda2e71ff9@live.com,0,1,7acb35b6-e1ec-44ed-9099-38580e330ed0,OneDrive
9,199.66.91.253,fc547e9e-f0c2-6000-1d75-39268176f5e6,2018-08-20T11:54:35,

### Databricks SQL AI

You can also talk to it using natural language via an agent

In [30]:
lui("get 4 events from o365-management-activity", agent="DatabricksAgent")

<LouieAI Notebook Interface | Session: Active | History: 2 responses | Traces: Disabled | Latest: 1 text>

### Louie Agent: Main Orchestrator Agent

Louie will pick appropriate agents and tools for you, so you do not need to specify them

In [31]:
lui("""get 4 events from databrick's o365-management-activity""")

,table_name
0,`client_demos`.`botsv3`.`access-combined`
1,`client_demos`.`botsv3`.`access_combined`
2,`client_demos`.`botsv3`.`alternatives`
3,`client_demos`.`botsv3`.`amazon-ssm-agent`
4,`client_demos`.`botsv3`.`amazon-ssm-agent-too-...
...,...
282,`system`.`information_schema`.`tables`
283,`system`.`information_schema`.`views`
284,`system`.`information_schema`.`volume_privileges`
285,`system`.`information_schema`.`volume_tags`


## Let's Vibe: Composing via DataFrames

In [32]:
lui("""

Get 100 events from databricks aws-s3-accesslogs and
draw a graph using any intereseting sounding columns for investigating

""")

<LouieAI Notebook Interface | Session: Active | History: 4 responses | Traces: Disabled | Latest: 2 text>

# Prompt Templates

### Reuse prompts as template files

Prompt template files and TODO/plan files are key to quickly using agents, having them do long runs, and increasing how much the AI manages itself.

Likewise, instead of manually editing plans, this helps enable you to tell the AI how to do that for you. As templates get more advanced, self-modifying plans become key.

In [34]:
lui_template = new_thread()

lui_template("Repeat this out: The best jokes are about 19th century scientists")

lui_template.thread_id

'D_6c7aAAe5X46lS7e7_mw5'

In [38]:
lui_joke = new_thread()

lui_joke(f"Read dthread {lui_template.thread_id}, and only after, make a joke about it")

<LouieAI Notebook Interface | Session: Active | History: 1 responses | Traces: Disabled | Latest: 3 text>

In [41]:
print(lui_joke.text)

I need to read the specified dthread first to understand its content before making a joke about it.
NotebookAgent
read dthread D_6c7aAAe5X46lS7e7_mw5Final Answer: Ah, 19th-century scientists! They were the original "mad scientists" before it was cool. Why did the 19th-century physicist bring a ladder to the bar? Because he heard the drinks were on the house, and he wanted to test the theory of relativity by getting
